In [ ]:
import numpy as np
import pandas as pd
import logging
from tqdm.auto import tqdm
import warnings
from typing import Union
warnings.filterwarnings('ignore')

## Data Preparation

Logger

In [ ]:
logger_ = logging.getLogger()
logger_.setLevel(1)

ch = logging.StreamHandler()
ch.setLevel(1)

logger_.addHandler(ch)

In [ ]:
def process(df):
    use_cols = ['_id', 'title', 'fos', 'abstract', 'lang', 'url', 'venue', 'authors', 'year',
                'volume', 'keywords', 'doi', 'references', 'n_citation']
    df = df[use_cols]
    df['authors'] = df['authors'].astype('str')
    df['references'] = df['references'].astype('str')
    df['keywords'] = df['keywords'].astype('str')
    df = df.drop_duplicates(subset=['title', 'authors', 'year', 'keywords', 'references', 'abstract'], keep='last')
    df['n_citation'].replace(np.nan, 0, inplace=True)
    df = df[(df['year'] < 2023)]
    df = df[(df['year'] > 1960) & (df['n_citation'] != 0)]
    return df

In [ ]:
chunksize = 5e5
sum = 0
i = 1
filenames = ['../input/cndbv13/test1.json', '../input/cndbv13/test2.json', '../input/cndbv13/test3.json', '../input/cndbv13/test4.json', '../input/cndbv13/test5.json']
for filename in filenames:  
    with pd.read_json(filename, lines=True, chunksize=chunksize) as reader:
        for chunk in tqdm(reader):
            df = process(chunk)
            new_filename = f'data_{i}.parquet'
            df.to_parquet(new_filename)
            i += 1

In [ ]:
df = pd.read_parquet('data_1.parquet')

## Train/Test split

Принцип разделения: статьи до определенного года - треин, после - тест.
Такой подход был выбран из-за следующих его преимуществ:
1. простота реализации метода
2. отстусвие коллизий, т.е пересечений в train/test
3. используется весь датасет после пред процессинга

Данный подход имел бы существенные недостатки, если бы паттерны написания статей и вектор исследований был бы существенно изменен. Такое действительно имеет место, но влиянием можно пренебречь, так как статьи до 1980-го года были исключены на этапе очистки данных.

In [ ]:
split_cfg = {
    'test_size': 0.15
}

In [ ]:
def get_most_suitable_year(test_size: float, 
                           data: pd.DataFrame,
                           years: set) -> int:
    "Just walk through all years and choose the most appopriate"
    dataset_size = len(data)
    year_desision, delta = -1, float('inf')
    for year in tqdm(years):
        cur_test_size = len(data[data.year > year])/ dataset_size
        if abs(cur_test_size - test_size) < delta:
            year_desision = year
            delta = abs(cur_test_size - test_size)
    return year_desision

In [ ]:
def train_test_split(data: pd.DataFrame) -> tuple:
    uniq_years = df.year.unique()
    board_year = get_most_suitable_year(split_cfg['test_size'],
                                        data,
                                        uniq_years)
    train, test = data[data.year <  board_year], data[data.year >  board_year]
    return (train, test)

In [ ]:
train_data, test_data = train_test_split(df)
assert len(train_data) > len(test_data)
logger_.info('Successfull data splitting')

Save result to csv..

In [ ]:
train_data.to_csv('/kaggle/working/train.csv')
test_data.to_csv('/kaggle/working/test.csv')